In [1]:
import torch
from transformers import pipeline

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

from PIL import Image
import matplotlib.pyplot as plt

2025-12-22 12:48:24.842949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766407705.034481      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766407705.089315      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_csv("/kaggle/input/mrbeast-youtube-comment-sentiment-analysis/sentiment_analysis_dataset.csv", sep=',', quotechar='"', engine='python', on_bad_lines='skip')

In [5]:
df.head()

,Comment,Sentiment
0,Achieving million views in days is dangerous,Positive
1,How many people here want to participate in su...,Neutral
2,Mrbeast is slowly turning into mrjigsaw,Negative
3,genuinely can't believe how dystopian this is,Negative
4,Have of the worlds smartest people compete in ...,Neutral


In [6]:
 df['Comment'] = df['Comment'].astype(str)

In [7]:
model_name_1 = "distilbert-base-uncased-finetuned-sst-2-english"
pipe_bert = pipeline("sentiment-analysis", model=model_name_1, device=-1)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [8]:
df = df.dropna(subset=['Comment'])

In [9]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
    
df['ProcessedComments'] = df['Comment'].apply(preprocess_text)
df[['Comment', 'ProcessedComments']].head()

,Comment,ProcessedComments
0,Achieving million views in days is dangerous,achieving million view day dangerous
1,How many people here want to participate in su...,many people want participate challenge
2,Mrbeast is slowly turning into mrjigsaw,mrbeast slowly turning mrjigsaw
3,genuinely can't believe how dystopian this is,genuinely cant believe dystopian
4,Have of the worlds smartest people compete in ...,world smartest people compete series intellect...


In [10]:
results_bert = pipe_bert(df['ProcessedComments'].tolist(), batch_size=16, truncation=True)

In [11]:
preds_bert = []
for res in results_bert:
    label = res['label'].lower() 
    preds_bert.append(label)

df['preds_bert'] = preds_bert

In [12]:
df.head()

,Comment,Sentiment,ProcessedComments,preds_bert
0,Achieving million views in days is dangerous,Positive,achieving million view day dangerous,negative
1,How many people here want to participate in su...,Neutral,many people want participate challenge,positive
2,Mrbeast is slowly turning into mrjigsaw,Negative,mrbeast slowly turning mrjigsaw,negative
3,genuinely can't believe how dystopian this is,Negative,genuinely cant believe dystopian,positive
4,Have of the worlds smartest people compete in ...,Neutral,world smartest people compete series intellect...,positive


In [13]:
df['Sentiment'] = df['Sentiment'].str.lower()
df['preds_bert'] = df['preds_bert'].str.lower()


In [14]:
print("Accuracy:", accuracy_score(df['Sentiment'], df['preds_bert']))
print("-" * 30)
print(classification_report(df['Sentiment'], df['preds_bert'], zero_division=0))

Accuracy: 0.5644754414601573
------------------------------
              precision    recall  f1-score   support

    negative       0.03      0.72      0.06        79
     neutral       0.00      0.00      0.00      1981
    positive       0.77      0.80      0.79      4679

    accuracy                           0.56      6739
   macro avg       0.27      0.51      0.28      6739
weighted avg       0.54      0.56      0.55      6739



Bad accuracy, let's check pipeline on comment without pre-proccessing.

In [15]:
results_bert = pipe_bert(df['Comment'].tolist(), batch_size=16, truncation=True)

In [16]:
preds_bert = []
for res in results_bert:
    label = res['label'].lower() 
    preds_bert.append(label)

df['preds_bert'] = preds_bert

In [17]:
df['Sentiment'] = df['Sentiment'].str.lower()
df['preds_bert'] = df['preds_bert'].str.lower()


In [18]:
print("Accuracy:", accuracy_score(df['Sentiment'], df['preds_bert']))
print("-" * 30)
print(classification_report(df['Sentiment'], df['preds_bert'], zero_division=0))

Accuracy: 0.6006825938566553
------------------------------
              precision    recall  f1-score   support

    negative       0.04      0.85      0.07        79
     neutral       0.00      0.00      0.00      1981
    positive       0.82      0.85      0.84      4679

    accuracy                           0.60      6739
   macro avg       0.29      0.57      0.30      6739
weighted avg       0.57      0.60      0.58      6739



this is better, but still not even close enough to SVM from lab 2 accuracy.

In [19]:

model_name_2 = "cardiffnlp/twitter-roberta-base-sentiment-latest"
pipe_roberta = pipeline("sentiment-analysis", model=model_name_2, device=-1)

results_roberta = pipe_roberta(df['Comment'].tolist(), batch_size=16, truncation=True)

mapping_roberta = {
    'label_0': 'negative', 'negative': 'negative',
    'label_1': 'neutral',  'neutral': 'neutral',
    'label_2': 'positive', 'positive': 'positive'
}

preds_roberta = []
for res in results_roberta:
    raw_label = res['label'].lower()
    clean_label = mapping_roberta.get(raw_label, raw_label)
    preds_roberta.append(clean_label)

df['pred_roberta'] = preds_roberta

print(f"\nРезультати {model_name_2}:")
print("Accuracy:", accuracy_score(df['Sentiment'], df['pred_roberta']))
print("-" * 30)
print(classification_report(df['Sentiment'], df['pred_roberta'], zero_division=0))

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Результати cardiffnlp/twitter-roberta-base-sentiment-latest:
Accuracy: 0.7780086066181926
------------------------------
              precision    recall  f1-score   support

    negative       0.09      0.73      0.15        79
     neutral       0.72      0.68      0.70      1981
    positive       0.91      0.82      0.87      4679

    accuracy                           0.78      6739
   macro avg       0.57      0.74      0.57      6739
weighted avg       0.85      0.78      0.81      6739



First model was for binary classification (oops) and that why it has a really poor performance. The second one is a bit better but still can't achieve SVM's 0.96 accuracy because we are using zero-shot classification.

# Zero-shot classification

Load multilingual bert:

In [20]:
classifier = pipeline(
    "zero-shot-classification", 
    model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Device set to use cuda:0


In [21]:
sequence_to_classify = "Голова Європейської Комісії оголосила про нові екологічні ініціативи щодо захисту лісів."
candidate_labels = ["економіка", "політика", "спорт", "екологія та природа"]

In [22]:
result_zsc = classifier(
    sequence_to_classify,
    candidate_labels,
    multi_label=False 
)

In [23]:
result_zsc

{'sequence': 'Голова Європейської Комісії оголосила про нові екологічні ініціативи щодо захисту лісів.',
 'labels': ['політика', 'екологія та природа', 'економіка', 'спорт'],
 'scores': [0.9518800973892212,
  0.04575416445732117,
  0.001624812139198184,
  0.0007409930694848299]}

In [24]:
for label, score in zip(result_zsc['labels'], result_zsc['scores']):
    print(f"- {label:<20}: {score:.4f}")

- політика            : 0.9519
- екологія та природа : 0.0458
- економіка           : 0.0016
- спорт               : 0.0007


# Summarization

In [25]:
summarizer = pipeline(
    "summarization", 
    model="google/mt5-small", 
    tokenizer="google/mt5-small"
)

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


In [26]:
long_ukrainian_text = (
    "Чат-боти, побудовані на великих мовних моделях (LLMs) типу Transformer, "
    "революціонізують взаємодію користувачів з технологіями. "
    "Вони здатні генерувати зв'язний, контекстуально релевантний та граматично правильний текст, "
    "завдяки навчанню на мільярдах слів. Цей прогрес дозволяє автоматизувати "
    "обслуговування клієнтів, створювати контент та допомагати в освіті. "
    "Однак, їх використання також викликає занепокоєння щодо етики, упередженості та достовірності інформації."
)

In [27]:
result_summarization = summarizer(
    long_ukrainian_text, 
    min_length=15, 
    max_length=40
)

Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [28]:
result_summarization[0]

{'summary_text': '<extra_id_0> користувачів зможуть отримати більше ніж мільярди слів.'}

# Translation

In [29]:
uk_to_en_translator = pipeline(
    "translation", 
    model="Helsinki-NLP/opus-mt-uk-en"
)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/305M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/305M [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [30]:
ukrainian_text_to_translate = "Машинне навчання є підмножиною штучного інтелекту, що дозволяє системам навчатися з даних."

In [31]:
result_translation = uk_to_en_translator(ukrainian_text_to_translate, max_length=100)

In [32]:
result_translation[0]

{'translation_text': 'Machine learning is a subset of artificial intelligence that allows systems to learn from data.'}